_Version Log: Starting to edit openai's convertion_

In [ ]:
import numpy as np
from scipy.fftpack import fft, idct
from l1magic import l1eq_pd

def dct_magic(k, m, n, b, Fs):
    A = np.zeros((m, n))
    for j in range(m):
        ek = np.zeros(n)
        ek[k[j]] = 1
        A[j, :] = idct(ek)

    y = np.linalg.pinv(A).dot(b)

    x = l1eq_pd(y, A, A.T, b, 5e-3, 32)

    xl2 = y

    l1 = x
    l1 = abs(l1) / max(abs(l1))

    xl1 = idct(x)

    kd = Fs / 2 * np.linspace(0, 1, n)

    return l1, xl1, kd

def ft_magic(k, m, n, b, Fs): #positions, *(shape of data array), signal (at position k), Niquist frequency 
    PHI = np.fft.fft(np.eye(n)) # numpy eye-denterty matrix
    S = np.zeros((m, n))

    for j in range(m):
        ek = np.zeros(n)
        ek[k[j]] = 1
        S[j, :] = ek

    A = S * (1/n) * PHI.T

    y = np.linalg.pinv(A).dot(b)

    x12 = (1/n) * np.real(PHI.T.dot(y))

    x = l1eq_pd(y, A, A.T, b, 5e-3, 32)

    l1 = 2 * x[:n//2]

    xl1 = (1/n) * np.real(PHI.T.dot(x))

    l1 = abs(l1) / max(abs(l1))

    kd = Fs / 2 * np.linspace(0, 1, n // 2)

    return l1, xl1, kd

def single_sided_ft(x, Fs, sin_theta_incident):
    N = len(x)
    X = fft(x)
    f = Fs / 2 * np.linspace(0, 1, N // 2)
    f = f[:, np.newaxis]
    X = 2 * np.abs(X[:N // 2])
    X = X / max(X)
    f = 1.0 / (f / (2 * sin_theta_incident))
    return X, f

def single_sided_ft_b(x, Fs, sin_theta_incident):
    N = len(x)
    X = fft(x)
    f = Fs / 2 * np.linspace(0, 1, N // 2)
    f = f[:, np.newaxis]
    X = 2 * np.abs(X[:N // 2])
    X = X / max(X)
    f = 1.0 / (f / (2 * sin_theta_incident))
    return X, f

# Now you can call the functions with your data and parameters.

In [ ]:
import numpy as np
from scipy.fftpack import fft, idct
import random

# Read in data
with open('150_points_500um_pp_0.25THz_width_287times2.txt', 'r') as fileID:
    data = np.loadtxt(fileID)
f = (data[:, 1] * 0.001).reshape(-1, 1) #f is our original data points
t = (data[:, 0] * 0.001).reshape(-1, 1) #t is the locations of the data points (x coordinate)
n = len(f) #n is the original number of data points
spacing = 0.25  # cm spacing

# Parameters to change
det_size = 120  # Detector size in points
position = 0.6 #central position of detector from 0 to 1

# Experiment conditions
dx = 500  # pixel pitch in um
Fs = 1.0 / dx # Niquist frequency
theta = 4  # Angle between beams
sin_theta_incident = np.sin(np.deg2rad(theta))
det_size_cm = det_size * dx * 1e-4  # Convert to cm
pnum = det_size_cm / spacing  # Every 0.5 cm

# If normal #to what???
m = int(np.ceil(pnum))

# The FFT of the input
spectrum_fft, um_fft = single_sided_ft(f, Fs, sin_theta_incident)

k_p = int(np.ceil(n * position))  # Find the centtral position of the detector
k_p_min = k_p - int(np.ceil(det_size / 2))  # starting position of detector
rand = random.sample(range(det_size), m)
k = np.array([x + k_p_min for x in rand])  # Sorry openai but acually, it is random.

if any(k > n):
    print('Detector is too big')
    break
elif any(k < 0):
    print('Detector is too small')
    break

b = f[k] # b is the undersampled version of f. k is the undersampled version of t.

# A = rows of DCT matrix with indices of the chosen points

A = np.zeros(b.shape)

l1, xl1, kd = ft_magic(k, *b.shape, b, Fs) #positions, shape of data array, signal (at position k), Niquist frequency 

b_fft_m, um_b_fft = single_sided_ft_b(b, Fs, sin_theta_incident)

um = 1.0 / (kd / (2 * sin_theta_incident))

# Plot
import matplotlib.pyplot as plt

t = t * (1.0 / 0.001)
plt.figure(1)
plt.subplot(3, 1, 1)
plt.plot(t, f, 'b', markersize=5)
plt.plot(t[k], b, 'or', markersize=12)
plt.title('f = signal, b = random sample')
plt.xlabel('length along detector cm')

plt.subplot(3, 1, 2)
GHz_fft = (1.0 / um) * 299.792458 * 1e3
plt.plot(GHz_fft, spectrum_fft, 'b-', GHz_fft, l1, 'r-')
plt.xlabel('GHz')
plt.axis([0, 500, 0, 1])


plt.title('x = l1 solution, A*x = b using FFT basis')

plt.subplot(3, 1, 3)
plt.plot(t, xl1, 'r-', t, f, 'b-')
plt.xlabel('length along detector cm')

if mode == 0:
    plt.title('DCT(x)')
else:
    plt.title('Reconstructed time domain signal from x = l1 solution')

plt.show()

# Write out
B = np.vstack((GHz_fft, l1))
with open('experimental_peaks.txt', 'w') as fileID:
    np.savetxt(fileID, B.T, fmt='%6.9f %12.9f')
